In [14]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
import numpy as np

In [41]:
loans = pd.read_csv('/content/drive/My Drive/ML Specialization/lending-club-data.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (19,47) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
loans.drop('bad_loans', axis='columns', inplace=True)

In [43]:
target = 'safe_loans'
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies
             'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]
           
# Extract the feature columns and target column
loans = loans[features + [target]].dropna()

In [44]:
loans.head()

,grade,sub_grade_num,short_emp,emp_length_num,home_ownership,dti,purpose,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,total_rec_late_fee,int_rate,total_rec_int,annual_inc,funded_amnt,funded_amnt_inv,installment,safe_loans
0,B,0.4,0,11,RENT,27.65,credit_card,8.14350,0.0,1.0,1.0,1,1,3.0,0.0,1.0,83.7,0.00,10.65,861.07,24000.0,5000,4975,162.87,1
1,C,0.8,1,1,RENT,1.00,car,2.39320,0.0,1.0,5.0,1,1,3.0,0.0,1.0,9.4,0.00,15.27,435.17,30000.0,2500,2500,59.83,-1
2,C,1.0,0,11,RENT,8.72,small_business,8.25955,0.0,1.0,2.0,1,1,2.0,0.0,1.0,98.5,0.00,15.96,603.65,12252.0,2400,2400,84.33,1
3,C,0.2,0,11,RENT,20.00,other,8.27585,0.0,1.0,1.0,0,1,10.0,0.0,1.0,21.0,16.97,13.49,2209.33,49200.0,10000,10000,339.31,1
4,A,0.8,0,4,RENT,11.20,wedding,5.21533,0.0,1.0,3.0,1,1,9.0,0.0,1.0,28.3,0.00,7.90,631.38,36000.0,5000,5000,156.46,1


In [46]:
categorical_variables = []
for feat_name, feat_type in zip(loans.columns, loans.dtypes):
    if feat_type == object:
        categorical_variables.append(feat_name)

for feature in categorical_variables:
    loans_data_one_hot_encoded = pd.get_dummies(loans[feature], prefix=feature, dummy_na=True)
    
    for column in loans_data_one_hot_encoded.columns:
        loans[column]= loans_data_one_hot_encoded[column]
    
    loans.drop(feature, axis='columns', inplace=True)

In [47]:
print(loans.columns)
print(len(loans.columns))

Index(['sub_grade_num', 'short_emp', 'emp_length_num', 'dti',
       'payment_inc_ratio', 'delinq_2yrs', 'delinq_2yrs_zero',
       'inq_last_6mths', 'last_delinq_none', 'last_major_derog_none',
       'open_acc', 'pub_rec', 'pub_rec_zero', 'revol_util',
       'total_rec_late_fee', 'int_rate', 'total_rec_int', 'annual_inc',
       'funded_amnt', 'funded_amnt_inv', 'installment', 'safe_loans',
       'grade_A', 'grade_B', 'grade_C', 'grade_D', 'grade_E', 'grade_F',
       'grade_G', 'grade_nan', 'home_ownership_MORTGAGE',
       'home_ownership_OTHER', 'home_ownership_OWN', 'home_ownership_RENT',
       'home_ownership_nan', 'purpose_car', 'purpose_credit_card',
       'purpose_debt_consolidation', 'purpose_home_improvement',
       'purpose_house', 'purpose_major_purchase', 'purpose_medical',
       'purpose_moving', 'purpose_other', 'purpose_small_business',
       'purpose_vacation', 'purpose_wedding', 'purpose_nan'],
      dtype='object')
48


In [48]:
import json
with open('/content/drive/My Drive/ML Specialization/module-8-assignment-1-train-idx.json', 'r') as json_file:
    train_indices = json.load(json_file)
with open('/content/drive/My Drive/ML Specialization/module-8-assignment-1-validation-idx.json', 'r') as json_file:
    valid_indices = json.load(json_file)

In [49]:
train_data = loans.iloc[train_indices]
valid_data = loans.iloc[valid_indices]

In [54]:
from sklearn.ensemble import GradientBoostingClassifier

model_5 = GradientBoostingClassifier(n_estimators=5, max_depth=6)
X = train_data.drop('safe_loans', axis='columns', inplace=False)
X = X.to_numpy()
Y = train_data['safe_loans'].to_numpy()
model_5.fit(X, Y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=5,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [55]:
validation_safe_loans = valid_data[valid_data[target] == 1]
validation_risky_loans = valid_data[valid_data[target] == -1]

sample_valid_data_risky = validation_risky_loans[0:2]
sample_valid_data_safe = validation_safe_loans[0:2]

sample_valid_data = sample_valid_data_safe.append(sample_valid_data_risky)
sample_valid_data

,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,total_rec_late_fee,int_rate,total_rec_int,annual_inc,funded_amnt,funded_amnt_inv,installment,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,grade_nan,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,purpose_nan
22,0.2,0,3,29.44,6.30496,0.0,1.0,0.0,1,1,8.0,0.0,1.0,93.9,0.0,9.91,823.48,92000.0,15000,15000,483.38,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
26,0.6,1,1,12.19,13.49520,0.0,1.0,0.0,1,1,8.0,0.0,1.0,59.1,0.0,11.71,1622.21,25000.0,8500,8500,281.15,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0
24,0.4,0,3,13.97,2.96736,3.0,0.0,0.0,0,1,14.0,0.0,1.0,59.5,0.0,16.77,719.11,50004.0,5000,5000,123.65,-1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0
41,1.0,0,11,16.33,1.90524,0.0,1.0,0.0,1,1,17.0,0.0,1.0,62.1,0.0,8.90,696.99,100000.0,5000,5000,158.77,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [58]:
X = sample_valid_data.drop('safe_loans', axis='columns')
print(model_5.predict(X.to_numpy()))
print(model_5.predict_proba(X.to_numpy()))

[ 1  1 -1  1]
[[0.41642331 0.58357669]
 [0.46949689 0.53050311]
 [0.53807792 0.46192208]
 [0.39591639 0.60408361]]


In [68]:
X = valid_data.drop('safe_loans', axis='columns')
Y = valid_data['safe_loans'].to_numpy()
print(model_5.score(X.to_numpy(), Y))
predictions = model_5.predict(X.to_numpy())
print('FP: ', np.sum((predictions==1) & (Y==-1)))
print('FN: ', np.sum((predictions==-1) & (Y==1)))

0.6613528651443343
FP:  1653
FN:  1491


In [74]:
valid_data['prob'] = model_5.predict_proba(X.to_numpy())[:,1]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [77]:
valid_data.sort_values('prob', ascending=False).head(5)

,sub_grade_num,short_emp,emp_length_num,dti,payment_inc_ratio,delinq_2yrs,delinq_2yrs_zero,inq_last_6mths,last_delinq_none,last_major_derog_none,open_acc,pub_rec,pub_rec_zero,revol_util,total_rec_late_fee,int_rate,total_rec_int,annual_inc,funded_amnt,funded_amnt_inv,installment,safe_loans,grade_A,grade_B,grade_C,grade_D,grade_E,grade_F,grade_G,grade_nan,home_ownership_MORTGAGE,home_ownership_OTHER,home_ownership_OWN,home_ownership_RENT,home_ownership_nan,purpose_car,purpose_credit_card,purpose_debt_consolidation,purpose_home_improvement,purpose_house,purpose_major_purchase,purpose_medical,purpose_moving,purpose_other,purpose_small_business,purpose_vacation,purpose_wedding,purpose_nan,prob
8021,0.4,0,4,12.73,12.16700,0.0,1.0,1.0,1,1,6.0,0.0,1.0,30.5,18.2281,5.99,1202.30,36000.0,12000,12000,365.01,-1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.673059
19815,0.6,0,11,2.40,2.49545,0.0,1.0,0.0,1,1,6.0,0.0,1.0,0.0,0.0000,6.17,671.39,110000.0,7500,7500,228.75,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0.661468
93680,0.4,0,6,15.39,6.14080,0.0,1.0,0.0,1,1,11.0,0.0,1.0,8.1,0.0000,6.62,891.43,120000.0,20000,19900,614.08,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.661468
112700,0.4,0,11,10.95,3.02852,0.0,1.0,0.0,1,1,18.0,0.0,1.0,14.5,0.0000,6.49,290.86,85000.0,7000,7000,214.52,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0.661468
104046,0.4,0,2,9.18,4.26159,0.0,1.0,1.0,1,1,19.0,0.0,1.0,17.3,0.0000,6.62,1008.74,83000.0,9600,9575,294.76,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0.661468


In [78]:
from sklearn.ensemble import GradientBoostingClassifier

model_10  = GradientBoostingClassifier(n_estimators=10, max_depth=6)
model_50  = GradientBoostingClassifier(n_estimators=50, max_depth=6)
model_100 = GradientBoostingClassifier(n_estimators=100, max_depth=6)
model_200 = GradientBoostingClassifier(n_estimators=200, max_depth=6)
model_500 = GradientBoostingClassifier(n_estimators=500, max_depth=6)

X = train_data.drop('safe_loans', axis='columns', inplace=False)
X = X.to_numpy()
Y = train_data['safe_loans'].to_numpy()
model_10.fit(X, Y)
model_50.fit(X, Y)
model_100.fit(X, Y)
model_200.fit(X, Y)
model_500.fit(X, Y)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=6,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=500,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)

In [81]:
X = valid_data.drop(['safe_loans', 'prob'], axis='columns')
Y = valid_data['safe_loans'].to_numpy()
print(model_10.score(X, Y))
print(model_50.score(X, Y))
print(model_100.score(X, Y))
print(model_200.score(X, Y))
print(model_500.score(X, Y))

0.6653382162860836
0.6820336062042223
0.6895734597156398
0.6895734597156398
0.6900043084877208
